# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

## 使用微调前 ChatGLM3

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

2024-02-18 21:53:23.044270: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-18 21:53:23.091129: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-18 21:53:23.091164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-18 21:53:23.092366: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-18 21:53:23.099696: I tensorflow/core/platform/cpu_feature_guar

In [7]:
print(response)

乾卦是《易经》中的第一卦，也是八卦之一。乾卦代表着天、强、积极、刚健等含义。乾卦是由两个阴爻夹一个阳爻构成，象征着天宇广阔、刚强有力、积极向上、充满生气。在《易经》中，乾卦象征着阳刚之气，具有领导地位，同时也象征着变化和进步。乾卦引导人们积极向上，充满生气，努力进取，永不放弃。


In [8]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
print(response)

地水师卦是《易经》中的第六卦，由两个阴爻夹一个阳爻构成，象征着地水相交，具有滋润、滋养、含蓄、耐心等含义。在《易经》中，师卦象征着智慧、谋略、策略、领导等含义。师卦引导人们深入思考，明辨是非，善于策划和领导，达到最终的成功。

师卦的含义还可以引申为教育、战争、领导等方面。在教育方面，师卦象征着教师的作用，教育学生，传授知识，启迪智慧。在战争方面，师卦象征着军队的指挥和战斗力，需要有计划、有策略、有领导才能取得胜利。在工作和领导方面，师卦象征着智慧和谋略，需要有耐心、有智慧、有领导才能取得成功。


## 微调前后效果对比

#### 加载 QLoRA Adapter(Epoch=3, Overfit, handmade-dataset)

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

### 使用微调后的 ChatGLM3-6B

In [10]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [11]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。它的卦辞是：“元、亨、利、贞”，预示着吉祥如意，同时也教导人们遵守天道的德行。乾卦所蕴含的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦的形状是：初爻为阳，后续七爻均为阳，象征着天地的广大无垠，以及万物养育之功。其时为夏，为阳极，因此象征着刚健强劲的特性。同时，这个卦象也预示着大吉大利，有利于实现目标。

在占卜中，干辣椒卦预示着大吉大利，得志如王。然而，需要注意的是，在这个过程中可能会遇到一些困难，因此需要谨慎应对，遵守天道。

在解卦时， Initial is the best, with the first爻 indicating the start of a new journey, and the following seven爻 representing the progress made during the journey. The time of the year is夏天， when the days are long and the weather is hot, representing the longest day of the year, symbolizing the long journey. The initial situation is unfavorable, but with caution and persistence, great achievements can be achieved.

在解卦时，需要结合具体的情况，结合乾卦的卦辞和卦象，结合时宜和运势，进行正确的理解和应用。


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？ 
乾卦是周易中的一卦，代表天，象征刚健强劲的特性。它由两个乾卦叠加而成，代表着天地之交，以及万物之始。在卜问中，乾卦预示着天圆、地广、万物充满活力，但同时也要警惕先前的查询，因为 answers might not be immediately obvious。在决策时，要刚强坚持正道，不能改变刚明的性格。在商业、事业中，刚柔并济，刚强刚直，能够取得成功。在人际关系中，要刚强坚贞，相信正义，能够结交到志同道合的朋友。

乾卦的

In [12]:
base_response, ft_response = compare_chatglm_results(query="地水师卦")

问题：地水师卦

原始输出：
{'name': '坤为地，代表大地，为众，为顺，为静，为柔，为育，为谦，为利，为美，为吉。', 'content': '\n坤卦的哲学是强调顺应自然，柔和谦逊，以追求和谐优美的生活方式。它代表了一种以柔克刚的哲学思维，强调顺应环境，顺应他人，以实现柔顺、和谐的结果。\n\n坤卦的符号是一幅由三条曲线相接组成的卦象，象征着大地的曲线，代表柔顺、和谐的特点。在卜问中，坤卦预示着顺从他人，以取得成功。\n\n坤卦的时运是柔，是众，是利，是美，是吉。这意味着在事业发展、社交互动、的情感关系等方面，应该以柔克刚为原则，注重顺应他人，以实现柔顺、和谐的结果。\n\n坤卦的运势是利于柔顺、和谐的生活，有利于众人的团结，有利于柔顺、温和的行事风格，有利于柔顺、和谐的人际关系。\n\n坤卦的解析是：柔和谦逊，以柔克刚，是实现和谐优美生活的关键。在事业发展中，要注重顺应他人，以取得成功。在社交互动中，要注重柔顺、和谐，以促进人际关系。在情感关系中，要注重柔和、温和，以促进感情和谐。'}


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦


卦象为：乾（天）为卦首，坤（地）为卦身，互为天地生，相互依存。

卦辞：天行地生，相互依存。

卦象：卦象为：乾（天）为卦首，坤（地）为卦身，互为天地生，相互依存。

卦辞：天行地生，相互依存。

卦断：断卦为断卦，即卦象所呈现的形势已经包含了一种必然的命运，无法改变。

天行地生，相互依存。

天行：指天象运行的规律，如风、云、雨、雾等。

地生：指地象运行的规律，如川、河、山、谷等。

卦辞中提到天行地生相互依存，这表示天地之间的相互影响和依存关系，也反映了人们在自然中的地位和命运。

卦象中，乾为天，代表刚强和刚毅，象征天象运行的规律；坤为地，代表柔顺和柔美，象征地象运行的规律。互为天地生，表示两者相互依存，相互影响。

卦辞中提到天行地生，相互依存，这意味着事物的发展变化是有其必然的规律，人们应该顺应这个规律，勇于接受挑战，勇于改变自己，从而获得成功。

总结起来，天行地生，相互依存， things are the way they are，人们应该顺应这个规律，勇于接受挑战，勇于改变自己，从而获得成功。


In [13]:
base_response, ft_response = compare_chatglm_results(query="天水讼卦")

问题：天水讼卦

原始输出：
在天水之间，时起时落的卦象 represents a period of transition and change in circumstances. This卦 requires the seeker to be adaptable and willing to change, as well as to be mindful of the consequences of their actions.

The keyword for this卦 is "change." This means that the seeker must be flexible and open to new ideas, as well as to be willing to let go of the past and move forward. The path of the卦 requires the seeker to be mindful of the consequences of their actions, as well as to be aware of the need for continuous learning and growth.

The ultimate goal of this卦 is to achieve harmony and balance in one's life, and to learn that change is a natural and inevitable part of life. The lessons of this卦 are important for building a strong and flexible mindset, and for developing the ability to adapt to new situations and circumstances.


ChatGLM3-6B微调后：
[gMASK]sop 天水讼卦


天山一柱倒地维，

陈希夷骑马入长安。

角行超骨力，

行路皆平川。

陈希夷：陈抟，唐代著名道士，传说中其能骑马入长安，又曾隐居在天水。

天山一柱倒地维：传说中，天山（今新疆地区）有一根柱子，名为地维，支撑着天空和大地。

角

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [15]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B (Epoch=3, automade-dataset) 微调后：\n{ft_response}")
    return base_response, ft_response

In [16]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，含有刚健之象。它由两个阳爻（象征刚）组成，它们相互配合，象征着天地的运行刚健不疲。乾卦的卦象为：初爻为阳，二爻为阳，三爻为阳，四爻为阳，五爻为阳，六爻为阳。这种连续阳爻的排列象征着天地的运行刚健不疲。

乾卦的含义是：天刚健刚强，运行不已，depend on the square and the circular. 

乾卦的时运是：利于开始，刚健不疲，有利于事务的发展。

乾卦的weather is: cloudy, windy, cold，dry.

乾卦的music is: tone of voice is sharp and clear, representing the courageous and determined spirit.

乾卦的clothing is: suit of clothes made of cloth, the color is鲜艳，decorative and ornate.

乾卦的color is: yellow or gold.

乾卦的message is: The message of the乾卦 is one of square and circular movements, representing the consistency and momentum of things. It emphasizes the importance of determination, perseverance and courage.

乾卦的utility is: It is useful for starting things, forging ahead, and for promoting growth.

乾卦的image is: The image of the天, with the sun at the center, representing the刚 and powerful symbol of the world.

乾卦的outlook is: Optimistic and confident, expecting the best outcome.

乾卦的personality is: Resolute, strong willed,

In [17]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是一个由三卦组成的卦象，代表天地之水，象征按着顺序瀑布冲淋，全体人力需在水师指挥下进行。在周易中，卦象被解释为阳刚之卦，预示着吉祥、顺利。

具体来说，每一卦都有不同的卦象，如乾为天，坤为地，震为雷，巽为风，坎为水，离为火，艮为山，兑为泽，卦象的组合则形成了各种不同的卦象，如乾为卦、乾为天，坤为地，坎为水，等等。

因此，地水师卦可以理解为在地的水中，有瀑布的形势，按照一定的顺序瀑布冲淋，人力需在水师指挥下进行。在周易中，地水师卦预示着吉祥、顺利，提示人们要谦逊、谦逊，接受天地的恩赐，在挫折中锻炼自己的意志，最终成功。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由三卦组成的卦象，代表天地之水，象征足涉水中。在卜问中，这个卦象表示困难中的人工，预示着事态会发展和变化。

具体来说，卦象是：乾（天）卦下，坎（水）卦上，离（火）卦中间。

这个卦象的寓意是，在困难中要坚持，不要灰心，最终会得到帮助。


In [18]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦（卦象为乾上坎下）是一种象征诉讼和争战的卦象。在周易的哲学中，讼卦代表着事物之间的矛盾和冲突，以及由矛盾和冲突导致的战争。讼卦的卦象象征着刚毅和刚强，但同时也提示着需要注意危险。在讼卦中，指数为3的卦象最为吉祥，表示刚强之举，但利于吉日，有利于 index 刚强，但需要谨慎从事，有利于指数ong治。

讼卦的时宜索引为：

1. 在讼卦中，指数为3的卦象最为吉祥，表示刚强之举，但利于吉日，有利于 index 刚强，但需要谨慎从事，有利于 index ong治。
2. 讼卦的初爻为乾，表示刚强，但刚中有柔，刚强而不至于独断独行，能够 indicators a，指数为6。
3. 讼卦的中爻为坎，表示刚强，但刚中有柔，能够 indicators a，指数为9。
4. 讼卦的末爻为地，表示刚强，但柔中带刚，能够 indicators a，指数为6。

讼卦的解卦为损卦（卦象为乾上坎下），代表刚强刚直，但物极则反，需要反转思维，谦逊冷静，吞纳 error。指数为6，象征刚强，但需要谨慎从事，有利于 index ong治。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 周易中的讼卦是什么卦

 讼卦是周易中的一卦，代表诉讼、争端和诉讼的卦象。它由两个卦象组成，上卦为乾（天），下卦为坎（水）。上卦的卦辞是“元”，下卦的卦辞是“亨”。两卦合并后，意味着“元亨”，表示事情顺利，吉利。

讼卦的时运是贞元时运，代表元亨，具有顺利成功的象征。在讼卦中，元凶之象特征明显，表示事情初始可能会有一些困难和挫折。然而，只要坚持到底，最终必定会成功。

讼卦的裁断是：先则观其象，后则观其变。初则以正，后则以变。利见大人，得志乃饮。

讼卦的卦象表示了诉讼的象象，预示着事情的顺利成功，但需要注意初始可能会有一些困难和挫折。坚持到底，最终必定会成功。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed))

In [19]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240218_214108"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [20]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：\n{ft_response}")
    return base_response, ft_response

In [21]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由两个乾卦叠加而成，代表天。它象征着君子 viewer 中的天，以及观卦者中的君子。乾卦的核心象征是：天行健，君子拟之。君子观此卦象, 从而知道如何行动。乾卦预示着观卦者将拥有坚定的信念和无私的奉献精神，能够排除困难，完成使命。', 'content': '\n乾卦包括六个阳爻，象征着天。它表示君子象天，君子观此卦象, 从而知道如何行动。乾卦预示着观卦者将拥有坚定的信念和无私的奉献精神，能够排除困难，完成使命。\n\n乾卦象征着天，它表示君子像天一样坚定、刚健、公正、勇敢、刚强。乾卦预示着观卦者将拥有坚定的信念和无私的奉献精神，能够排除困难，完成使命。'}


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [22]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
{'name': '履卦是由下卦乾和上卦坎组成的。坎象征着云，乾象征着天。云聚于天，形成了云彩的卦象。在这个卦象中，作者警告君子要履着卦中的刚柔结合的特性，以获取吉祥。', 'content': '\n履卦的卦象象征着Cloud over the Wind (云淡风清)。根据《易经》，云淡风清的卦象预示着 Cloudy Water (云泽) 之吉, but also a warning of Cloud Burst (云爆发) 之凶。\n\n在事业方面，云淡风清的卦象预示着君子们需要履着卦中的刚柔结合的特性，获取吉祥。但是，也容易引发云爆发之凶，因此，需要谨慎行动，密切应对。在事业上，需要适应时势，密切应对，方能获得吉祥。\n\n在经商方面，云淡风清的卦象预示着 Cloudy Water (云泽) 之吉，但也需要注意避免云爆发之凶。因此，经商者需要冒险，但务必谨慎行事，避免损失。 also, it is a warning of a possible financial crisis.\n\n在婚恋方面，云淡风清的卦象象征着吉祥的婚姻。'}


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 地水师卦是什么？ 师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


In [23]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
干的卦象是由乾卦（天）下，坤卦（地）上组成的。它代表着大吉大利，预示着君子多闪现在他人。大吉大利，独占鳌头，吉祥环绕，吉祥如春，天地相应，大吉大利。然而，由于讼卦是由两个卦象组合而成，因此，它也代表着忧愁和困难，如道路崎岖，艰难困苦。但是，君子劳苦功高，能苦心经营，终于得到了回报。因此，讼卦劝人努力奋斗，耐心经营，充满希望，带来大吉大利。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是周易中的一卦，由上卦乾（天）和下卦坎（水）组成。这一卦象代表着天地相交，代表着天地之间的 conflict，也象征着天地之间的和解。在这个卦象中，君子们会相互和解，君子们也会相互boxing，boxing意为拳击，即打斗。虽然讼卦代表着和解，但是长期的诉讼会带来不幸。虽然君子们会和解，但是和解后也会产生 backlog，即积压。虽然卦象吉祥，但是务必谨慎行事。 在 business 领域中，经商的人需要谨慎经营，避免冲突，寻求和解，才能获得好运。此外，对于和解, 也必须谨慎，避免过于草率。在人际关系中，虽然朋友之间可以和解，但是也必须注意避免发生冲突。对于恋爱，也会有一定的和解机会，但必须谨慎行事。
